# MF with ALS Collaborative Filtering

Create a basic matrix factorization (MF) with alternating least squares (ALS) collaboritive filtering solution (mf-als) for recommended tracks. 
This is built from the Verstrepen2017 matrix notation of a score matrix factored by a user embedding and item embedding matrix.

    Score = User-embedding x Item-embedding 
    
This is implemented using the techniques [using the implicit library](https://towardsdatascience.com/alternating-least-square-for-implicit-dataset-with-code-8e7999277f4b).

The matrix factorization is built from an append of the challenge set to the training set.
This is required to ensure all playlists (pids, a.k.a. users) are represented in the model and it can be used to make recommendations for the 

The cold start task is address with a simple global popularity ranking of tracks.
We compute the global popularity of tracks and recommend the first 500 in ranked order.
This set is also included as backfill to each playlist recommendation accross all subtasks to ensure the minimum 500 tracks are available to each challenge solution.

In [ ]:
import sys
import json
import re
import collections
import os
import datetime
import pandas as pd
import numpy as np
from datetime import date
import implicit

## Set parameters for Run

These parameters control the selection of dataset, challenge set, and solution output.

Must select dataset and challege_name.  The rest are derived parameters and can be left alone.

In [ ]:
# path to the directory with different collections of training and test sets
datadir="data/"  # path to base data set

# select the data for training  
#dataset="mpd/data"     # the original full mpd training set, requires quick=True and max_files set
#dataset="mpd-1st-21k"  # first 21 files of mpd, equiv to quick=True and max_files 20
#dataset="mpd-2nd-21k"  # second 21 files of mpd, avoids using data that built mympd challenge set
dataset="mympd-full-20k"

# select data for testing, this is the challenge_set.json file for specific challenge data
#challenge_name="mpd"  # original mpd challenge set, use with aicrowd
#challenge_name="mympd"  # my custom challenge set for task analysis
challenge_name="mympd-full"  # my custom challenge sampled from full training set for task analysis

# derived parameters, no need to change.

#trainset="data/mpd/data"  # relative path to training set in slice json file format
trainset=datadir+dataset+"/data"  # relative path to training set in slice json file format

#testset="data/challenge_set.json" # relative path to test set in challenge_set.json format
#testset=datadir+challenge_name+"-challenge-set/challenge_set.json" # relative path to test set in challenge_set.json format
testset=datadir+challenge_name+"/challenge_set.json" # relative path to test set in challenge_set.json format

datestr=date.isoformat(date.today())

challenge_header = "team_info,jprorama,jprorama@gmail.com\n"
challenge_solution = "method-mfals-"+challenge_name+"-"+dataset+"-"+datestr+".csv"

In [ ]:
# generate more verbose output for some functions
debug = True

# parameters for mpd load, superceeded by mpd-1st-21k
quick = False
max_files_for_quick_processing = 20

# random state
seed = 1

## Load the mpd slice files

Create one big data frame to make it simple to select the random samples.

In [ ]:
playlists = pd.DataFrame()
tracks = pd.DataFrame()

In [ ]:
def process_mpd(path):
    global playlists, tracks;
    
    count = 0
    filenames = os.listdir(path)
    for filename in sorted(filenames):
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            if debug: print("loaded {}:".format(fullpath))
            mpd_slice = json.loads(js)
            # Flatten data
            # extract slice info to keep association with original training files.
            slice_info = mpd_slice['info']['slice']
            slice_playlists = pd.json_normalize(mpd_slice, record_path=['playlists'])
            slice_playlists["slice"] = slice_info
            if debug: print("slice length {}:".format(len(slice_playlists)))
            slice_tracks = pd.json_normalize(mpd_slice['playlists'], record_path=['tracks'], meta=['pid'])
            # drop tracks from playlist dataframe
            # not worth it to save space, just makes it harder to reconstruct the playlist
            #slice_playlists.drop(columns='tracks', inplace=True)
            playlists = playlists.append(slice_playlists)
            tracks = tracks.append(slice_tracks)
            count += 1

            if quick and count > max_files_for_quick_processing:
                break


In [ ]:
%%time
process_mpd(trainset)

Set a new index for playlists so each row has unique id using pid. After reading the slice files the index values repeat for each slice.

Preference is to not use the pid since that drops this data column.
Instead create a new column of integers for each row and then set that as the index.

In [ ]:
playlists["newidx"]=range(len(playlists))

playlists.set_index("newidx", inplace=True)

In [ ]:
playlists

In [ ]:
pl = playlists.copy()

In [ ]:
pl = playlists[["pid","tracks"]].explode("tracks")

In [ ]:
pl["track_uri"] = [d.get("track_uri") for d in pl.tracks]

In [ ]:
pl["artist_name"] = [d.get("artist_name") for d in pl.tracks]

The expanded one-row-per-track representation shows we have 1.4million songs (rows). The row index has 21k entries which matches the 21k playlists in the training set.

In [ ]:
pl

### Check memory usage

In [ ]:
tracks = pl[["track_uri"]]

In [ ]:
tracks.memory_usage(deep=True)

In [ ]:
tracks.memory_usage()

In [ ]:
pd.__version__

In [ ]:
tracks.info()

From example in [pandas sparse data types page](https://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html) use memory_usage().sum().  Not clear why we divide by 1000.  Would think that makes it kilobytes.

In [ ]:
'dense : {:0.2f} kbytes'.format(tracks.memory_usage().sum() / 1e3)

## One Hot encode playlists

Attempting to use get_dummies() works in the dense space an tries to build a dataframe of 100k by 1.4Million songs.  Not sure why so many rows but it's still to big for ram at 300+G

trackhots = pd.get_dummies(tracks, dtype=bool)

sklearn has a onehot encoder that is a preprocessor to many of its routines.  See if we can fit the tracks to this representaiton.

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer


In [ ]:
pl[["pid", "track_uri"]].info()

In [ ]:
trackhots = OneHotEncoder()

In [ ]:
trackhots.fit(pl[["pid", "track_uri"]])

In [ ]:
trackhots.categories_

In [ ]:
trackhots.get_feature_names()

Transform the original data into a matrix representation.

Here again is the 1.4x290k represenation.  The 1.4k is the songs, so rows in the original matrix but not clear where the 290k comes from.  Would expect 21k for the playlists.

In [ ]:
th = trackhots.transform(pl[["pid","track_uri"]])

In [ ]:
th

In [ ]:
pl["pid"].max()

In [ ]:
playlists["num_tracks"].sum()

Hmm, there are some problems in the transformation.  The 1.4mil comes from the total number of tracks in training.  The total unique is much smaller.

In [ ]:
pl["track_uri"].drop_duplicates().count()

I'd expect an transformed data set to be 21k by 269k.

Ah, the onehot encoder wants a feature set of each record with its distinct features.
https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-categorical-features

in this case it's rows of track_uri.
so each row with mapp to the idx value and will just have tracks.

In [ ]:
playlists.head(1)

In [ ]:
import ast

In [ ]:
pl[["track_uri"]]

Try converting each tracks string to a data type 

https://www.geeksforgeeks.org/python-convert-string-dictionary-to-dictionary/

In [ ]:
playlists[["tracks"]].tracks

We need a list of lists. This is pretty easy to construct with a list comprehension to wrap the lists into a list.

In [ ]:
pltracks = [d for d in playlists[["tracks"]].tracks.apply((lambda s: [d["track_uri"] for d in s]))]

In [ ]:
len(pltracks)

In [ ]:
type(pltracks)

what we are really trying to do is train the encoding and then transform each row.

this is more like having a vocabulary and different sentances.
I need to map each sentance to it's onehot encoding of the vocabulary.

this example shows moving from an integerencoding to a one hot encoding
https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/

reading the docs leads to multi label binarizer which appears to be closer to what i want.
https://scikit-learn.org/stable/modules/preprocessing_targets.html#multilabelbinarizer

In [ ]:
mlb = MultiLabelBinarizer(sparse_output=True)

In [ ]:
pltracks = mlb.fit_transform(pltracks)

We finally have a list of 21k playlists encoded with the 269k unique tracks.`

In [ ]:
pltracks

## Append Challenge Set to Training Set

Read the data from the challenge set file and append the tracks to training tracks in prep for similarity comparison. Omit first 1000 tracks since this is the title only subtask. Their similarity is implicitly zero on all tracks

In [ ]:
# load data using Python JSON module
with open(testset,'r') as f:
    data = json.loads(f.read())

In [ ]:
# Flatten data
challenge_playlists = pd.json_normalize(data, record_path=['playlists'])

In [ ]:
[challenge_playlists["tracks"]]

In [ ]:
chtracks = [d for d in challenge_playlists[["tracks"]].tracks.apply((lambda s: [d["track_uri"] for d in s]))]

In [ ]:
chtracks[1002]

In [ ]:
pltracks = [d for d in playlists[["tracks"]].tracks.apply((lambda s: [d["track_uri"] for d in s]))]

In [ ]:
pltracks[0]

In [ ]:
chtracks[1000]

In [ ]:
alltracks = list()

In [ ]:
alltracks = pltracks + chtracks[1000:]

In [ ]:
len(alltracks)

## Compute MF of train and challenge tracks

The data set is combined with training and challenge to compute the missing track score, i.e. recommendations.

In [ ]:
allmpb = mlb.fit_transform(alltracks)

In [ ]:
allmpb

Use the hyper parameters from Volksov2018.
"we found that using rank 200 with α = 100 and λU = λV = 0.001 produced good performance"

Scale all binary indicators by 100 to introduce the confidence weight of

c_ui=1+\alpha(R_ij)

This multiplcation aproach doesn't seem to get the 1 in place so not clear how ALS compute i the library.


In [ ]:
allmpb_weighted = allmpb.multiply(100)

In [ ]:
allmpb_weighted = allmpb.multiply(100)

In [ ]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=200, regularization=0.001)


In [ ]:
model.fit(allmpb_weighted.T)

In [ ]:
# recommend items for a user
#user_items = item_user_data.T.tocsr()
recommendations = model.recommend(21000, allmpb, N=500)

In [ ]:
recommendations

In [ ]:
[id for id, score in model.recommend(21000, allmpb, N=500)]

## Explore sorting by score

In [ ]:
import itertools

In [ ]:
#from itertools import izip

def sort_csr(m):
    tuples = zip(m.indices, m.data)
    return sorted(tuples, key=lambda x: (x[1]), reverse=True)


## Compute Top-N Tracks

Use the global top-N tracks to backfill missing data for playlist recommendation.
The top-n global also forms a baseline recommender to assess overall performance.

We focus on the global track stats for the training dataset only. It makes sense because this is the known data but also because the final submission requires removal of challenge seed tracks so any track information learned from the challenge set can't be reused in the same list. Worth questioning some since it can't be reused in the same seed but could be in other playlists...

However, do we need the full corpus so that we can match up the vocabulary terms to the tracks?  No because we can just use the Vecotrizors vocab.

Approach is to treat the training playlists as corpus of text documents.  Count the occurance of terms with the count vectorizer.  Then sum the term counts into a single array.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix

Consruct corpus from playlists joined a list of strings.

In [ ]:
[ " ".join(doc) for doc in alltracks[0:2]]

We use a custom tokenizer to avoid splitting on the punctuation characters in the "spotify:track:xxx' pattern. https://stackoverflow.com/a/37884104/8928529

In [ ]:
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(' '))

In [ ]:
trackcounts = vectorizer.fit_transform([" ".join(doc) for doc in alltracks[0:20000]])

In [ ]:
trackcounts

In [ ]:
pd.DataFrame.sparse.from_spmatrix(trackcounts)

In [ ]:
trackfreq = trackcounts.T.sum(axis=1)

In [ ]:
len(trackfreq)

In [ ]:
trackfreq.shape

In [ ]:
pd.DataFrame(trackfreq)

In [ ]:
trackfreq

In [ ]:
dtype = [("trackidx", int), ("count", int)]
new = np.empty(len(trackfreq), dtype)

In [ ]:
new

In [ ]:
np.array([*range(len(trackfreq))]).astype(int)

In [ ]:
new['trackidx'] = np.array([*range(len(trackfreq))])

In [ ]:
np.array(trackfreq).reshape(len(trackfreq)).shape

In [ ]:
new['count'] = np.array(trackfreq).reshape(len(trackfreq))
#print new

In [ ]:
new

In [ ]:
pd.DataFrame(new)

In [ ]:
tuples = zip(new["trackidx"], new["count"])

In [ ]:
topn = sorted(tuples, key=lambda x: (x[1]), reverse=True)

In [ ]:
topn

In [ ]:
vectorizer.vocabulary_

Invert the vocabulary dictionary so we can map it to the tracks. https://stackoverflow.com/a/483833/8928529

In [ ]:
inv_map = {v: k for k, v in vectorizer.vocabulary_.items()}

In [ ]:
inv_map

Generate a list of the topn track identifiers

In [ ]:
toptracks=[inv_map[i] for i, cnt in iter(topn)]

In [ ]:
toptracks = toptracks[0:1000]

The top tracks can now be appended to each list an ensure we have the minimum recommenadable track set.

## Explore filtering challenge tracks from recommendation list

Remove the challenge tracks from the recommendation list.  The recommendation list is augmented with the topn (n=1000) most popular tracks to backfill recommendations that don't have enough tracks to fill the 500 count requirement.

The algorithm slows noticably as the number of challenge tracks increases and the recommendation list has to be search repeatedly.

The 9000 scored playlists start for playlist 1000.
The challenge playlist starts with the first playlist.
Need to offset the challenge playlist index to match the score structure and recommended tracks.

In [ ]:
reclist = list()
indexdist = list() #pd.DataFrame(columns=["index"])
misses = 0
tooshort = 0
trace = False

for idx in range(9000):
    # get the candidate track class id sorted by recommendation score
    cantracks = model.recommend(idx+21000, allmpb, N=500) # [id for id, score in model.recommend(idx, allmpb, N=500)]
    # convert class id to spotify track name
    rectracks=[mlb.classes_[i[0]] for i in cantracks]
    # ensure mininum length recommendation meets 500 tracks requirement
    rectracks=rectracks + toptracks
    if (trace): print("idx {}:".format(idx))
    # remove challenge tracks from recommendation list
    # note: the challenge tracks start at index 1000 to allign tasks with recommendations
    for challenge_track in chtracks[idx+1000]:
        if (trace): print("look for track: {}".format(challenge_track))
        while challenge_track in rectracks:
            try:
                indexdist.append(rectracks.index(challenge_track))
                if (trace): print("remove track pos: {}".format(rectracks.index(challenge_track)))
                rectracks.remove(challenge_track)
            except (ValueError, AttributeError):
                if (trace): print("didn't find in rectracks: {}".format(challenge_track))
                misses += 1
    #if reclist < 500:
    #    tooshort += 1
    
    # truncate recommendation list to the 500 length required
    reclist.append(rectracks[0:500])
    
    # progress bar
    if (idx % 1000) == 0: print("challenge tracks progress: {}".format(idx))


The index distribution is a simple peek into the performance of the KNN algorithm.  It shows the average index value of the seeded challenge tracks found in the recommendations.  Given that this number is dominated by values above the general playlist lengths and even above 500 it's clear that the similarity measure alone is not an effective way of organizing the recommendation results.

Correction: the challenge set index was misalligned with the recommendation set index.  After updating the index to start after the title only task (+1000) the collection of matches shifted to where 70% of tracks were found in the first 500 recommendations.

indexdist = pd.DataFrame(indexdist)

The distribution of removals shows that the vast majority are well above the 500 reclist limit

indexdist.describe()

In [ ]:
len(reclist)

In [ ]:
misses

In [ ]:
len(reclist[8995])

In [ ]:
submission = pd.DataFrame(reclist)

In [ ]:
submission

# add the top popular to flesh out recommendation

In [ ]:
toptracks[0:5]

In [ ]:
coldstart = pd.DataFrame(columns=[*range(500)])


In [ ]:
coldstart

In [ ]:
coldstart = coldstart.append([toptracks[0:500]])

In [ ]:
coldstart

In [ ]:
# create data frame of 1000 copies of top popular to substitute for title only cold start recommendation

coldstart = pd.DataFrame(columns=[*range(500)])

for i in range(1000):
    coldstart = coldstart.append([toptracks[0:500]])

In [ ]:
coldstart.shape

In [ ]:
solution = coldstart.append(submission)

In [ ]:
solution.shape

Reset the index so all rows have a distinct index value.  This is necessary so the pid insert below correctly adds each pid to the title only task.  Otherwise the repeated rows of the task are seen as a single distinct row and all get the same pid.

In [ ]:
solution = solution.reset_index(drop=True)

## Add playlist id into the submission

In [ ]:
[*range(10,10,1)]

generate playlist ID value range to create correct submission format.  We don't want to use a naive range of ids like the following

    pid = pd.DataFrame([*range(1000000,1010000)]) 

We want to use the original pids from the challenge set.
The pid is not arbitrary and should match the order of the pid in the challenge_playlists.  
The rows of the solution are in the order of the original challenge set to should apply directly.

In [ ]:
pid = challenge_playlists["pid"]

In [ ]:
pid=pid.to_frame()

In [ ]:
pid.shape

In [ ]:
pid

In [ ]:
solution.insert(0, "pid", pid) #[*range(2000000,2010000)])

In [ ]:
solution#[998:1005]

## Write solution to output file

Include only the data not any index or headers.

In [ ]:
solution_csv = solution.to_csv(index=False,header=False)

In [ ]:
text_file = open(challenge_solution, "w")
n = text_file.write(challenge_header+solution_csv)
text_file.close()